# 4 - analyze_creators

Este notebook realiza análises sobre os criadores de conteúdo a partir das tabelas `default.users_yt` e `default.posts_creator`.  
O fluxo inclui:  
- filtro dinâmico para considerar apenas os últimos 6 meses de publicações;  
- cálculo dos **Top 3 posts por Likes** e dos **Top 3 posts por Views** de cada criador (com desempate por data de publicação);  
- identificação de usuários presentes em `posts_creator` mas ausentes em `users_yt`;  
- contagem de publicações por mês ao longo de todo o período;  
- construção de uma grade completa (criador × mês) exibindo **0** nos meses sem posts;  
- transformação final em formato pivotado, com uma coluna para cada mês.  

O resultado fornece uma visão abrangente da produção de cada criador, tanto em termos de performance (likes/views) quanto de frequência (posts por mês).  




## Obs. o código considera 6 meses para trás a partir da data da última postagem (published_date = 2024-04-15 - conforme query abaixo) 
já que na tabela default.posts_creator não há postagens nos últimos 6 meses voltando da data de hoje, então as primeiras duas respostas (de top 3) e o exercício extra 2 retornariam apenas zeros.



In [0]:
%sql
SELECT 
    DATE(published_at_ts) AS published_date
FROM default.posts_creator
ORDER BY published_at_ts DESC
LIMIT 1

In [0]:
# Imports and Data Loading
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import DataFrame

users_df = spark.table("default.users_yt")
posts_df = spark.table("default.posts_creator")


# Dynamic 6-month limit based on published_month (DATE)
month_bounds = posts_df.agg(F.add_months(F.max("published_month"), -6).alias("six_months_limit_month"))

# Filter posts where month >= limit (keeps everything distributed)
posts_recent = (
    posts_df.crossJoin(month_bounds)
        .filter(F.col("published_month") >= F.col("six_months_limit_month"))
)

# Analysis base: join with users_yt (broadcast for performance)
base_analysis_df = (
    posts_recent.join(F.broadcast(users_df), posts_recent["yt_user"] == users_df["user_id"], "inner")
)

# Top 3 by LIKES (tie-breaker by published_at_ts desc)
w_likes = Window.partitionBy("user_id").orderBy(F.col("likes").desc(), F.col("published_at_ts").desc())
top3_likes_df = (
    base_analysis_df
      .withColumn("rank", F.row_number().over(w_likes))
      .filter(F.col("rank") <= 3)
      .select("user_id", "title", "likes", "rank")
      .orderBy("user_id", "rank")
)

# Top 3 by VIEWS (tie-breaker by published_at_ts desc)
w_views = Window.partitionBy("user_id").orderBy(F.col("views").desc(), F.col("published_at_ts").desc())
top3_views_df = (
    base_analysis_df
      .withColumn("rank", F.row_number().over(w_views))
      .filter(F.col("rank") <= 3)
      .select("user_id", "title", "views", "rank")
      .orderBy("user_id", "rank")
)

# Users in posts_creator that are NOT in users_yt
missing_users_df = (
    posts_df.select("yt_user").distinct()
        .join(users_df.select("user_id"), F.col("yt_user") == F.col("user_id"), "left_anti")
)

# Posts per month (entire period)
posts_per_month_df = (
    posts_df
      .groupBy("yt_user", "published_month")
      .count()
      .withColumnRenamed("published_month", "month") 
)


# Complete grid (0 for months without videos), covering the entire DF period
date_bounds = posts_df.agg(
    F.min("published_month").alias("first_month"),
    F.max("published_month").alias("last_month"),
)

months_df = (
    date_bounds
      .select(F.expr("sequence(first_month, last_month, interval 1 month) AS months"))
      .select(F.explode("months").alias("month"))
)

creators_df = posts_df.select("yt_user").distinct()
full_grid_df = creators_df.crossJoin(months_df)

posts_complete_df = (
    full_grid_df.join(posts_per_month_df, ["yt_user", "month"], "left")
        .select(
            F.col("yt_user").alias("user_id"),
            F.col("month"),
            F.coalesce(F.col("count"), F.lit(0)).alias("posts_count")
        )
)

# Pivot: columns per month
pivoted_df = (
    posts_complete_df
      .withColumn("month_ym", F.date_format("month", "yyyy-MM"))
      .groupBy("user_id")
      .pivot("month_ym")
      .sum("posts_count")
      .orderBy("user_id")
)




## 1) Top 3 posts por likes nos últimos 6 meses, por creator

In [0]:
display(top3_likes_df)

## 2) Top 3 posts por views nos últimos 6 meses, por creator

In [0]:
display(top3_views_df)


## 3) YT users em posts mas não em users_yt

In [0]:
display(missing_users_df)


## 4) Quantidade de publicações por mês, por creator

In [0]:
display(posts_per_month_df.orderBy("yt_user", "month"))


## Exercício extra 1: Mostrar 0 para meses sem vídeo

In [0]:
display(posts_complete_df.orderBy("user_id", "month"))


## Exercício extra 2: Pivot — colunas por mês

In [0]:
display(pivoted_df)